### Convert PDF files into CSV file

In [1]:
import pandas as pd
import numpy as np

In [2]:
from tabula import read_pdf

### 20152016, 20172018

In [3]:
pdf_paths = ['pdfs/Per Month per Grade - 2015 16.pdf', 'pdfs/Per Month per Grade-20172018.pdf']

In [4]:
def clean_and_append(df):
    # Forward fill month data
    df['Month'] = df['Month'].ffill()

    # Select relevant rows with coffee grades AA, AB, and C
    df = df.loc[df['Grade'].isin(['AA', 'AB', 'C'])]

    try:
        # Reorder relevant columns
        df = df[['Month', 'Grade', 'Avg']]
    except:
        df = df[['Month', 'Grade', 'Price']]

    with open('monthly_coffee_prices.csv', 'a') as f:
        df.to_csv(f, header=False)


In [5]:
def clean_and_append_special_2017(df):
    df['Month'] = df.apply(add_month, axis=1) 
    df['Month'] = df['Month'].ffill()

    df['Price'] = df.apply(extract_price, axis=1)
    
    df.columns = df.columns.astype(str)
    df = df.loc[df['0'].isin(['AA', 'AB', 'C'])]
    df = df[['Month', '0', 'Price']]
    
    with open('monthly_coffee_prices.csv', 'a') as f:
        df.to_csv(f, header=False)
        
    return df

In [6]:
def clean_and_append_special_2015(df):
    df['Month'] = df.apply(add_month, axis=1) 
    df['Month'] = df['Month'].ffill()

    df['Price'] = df.apply(extract_price, axis=1)
    df['Grade'] = df.apply(extract_grade, axis=1)
    
    df.columns = df.columns.astype(str)
    df = df.loc[df['Grade'].isin(['AA', 'AB', 'C'])]
    df = df[['Month', 'Grade', 'Price']]
    
    with open('monthly_coffee_prices.csv', 'a') as f:
        df.to_csv(f, header=False)
        
    return df 

In [7]:
for pdf_file in pdf_paths:
    try:
        df = read_pdf(pdf_file, pages='all')
        
        # Reset header
        df.columns = np.concatenate([df.iloc[0, :2], df.columns[2:]])
        df = df.iloc[1:].reset_index(drop=True)
        
        clean_and_append(df)
        
    except Exception as e:
        print('Error with pdf file: ', pdf_file)
        print(e)


## Special cases

In [8]:
def add_month(row):
    val = row[0]
    if len(val) > 3:
        try:
            # Check for year
            val_arr = val.split(' ')
            check_year = val_arr[1]
            if check_year.startswith('20'):
                return val
            else:
                return None
        except:
            return None
    else:
        return None

In [9]:
def extract_price(row):
    vals = str(row[5])
    vals_arr = vals.split(' ')
    
    result = None
    try:
        result = vals_arr[1]

    except:
        result = None
    
    return result

In [10]:
def extract_grade(row):
    vals = str(row[0])
    
    try:
        vals_arr = vals.split(' ')

        if len(vals_arr[0]) <= 2:
            return vals_arr[0]
        else:
            return None
    except:
        return None

### 20142015 

20142015 PDF is not so straightforward, needs special handling

In [11]:
df_2015 = read_pdf('pdfs/Per Month per Grade-20142015.pdf', pages='all', multiple_tables=True)

In [12]:
df = df_2015[1]

In [13]:
df

,0,1,2,3,4,5,6,7
0,Date By,NaN,Bags,Weight,Min,Max,VALUE,Avg
1,Month,Grade,Bought,Bought,PRICE,PRICE,US Dollar,PRICE
2,NaN,UG,640,"38,340",184,185,"141,486.60",184.52
3,NaN,UG1,"1,703","104,082",102,220,"406,908.18",195.47
4,NaN,UG2,995,"61,746",77,196,"174,871.64",141.61
5,NaN,UG3,11,704,167,167,"2,351.36",167.00
6,Month Total,NaN,"38,519","2,354,086",41,418,"10,775,767.72",228.87
7,January 2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,AA,"8,164","497,711",118,486,"2,899,413.22",291.27
9,NaN,AB,"19,723","1,194,270",116,400,"5,608,204.82",234.80


In [14]:
# Last 3 tables of df_2015 are well formed.
### First
df = df_2015[1]
df.columns = ['Month', 'Grade', 'Bags', 'Weight', 'Min', 'Max', 'USD', 'Price']
clean_and_append(df)

### Second
df = df_2015[2]
df.columns = ['Month', 'Grade', 'Bags', 'Weight', 'Min', 'Max', 'USD', 'Price']
clean_and_append(df)

### Third
df = df_2015[3]
df.columns = ['Month', 'Grade', 'Bags', 'Weight', 'Min', 'Max', 'USD', 'Price']
clean_and_append(df)

In [15]:
df = df_2015[0]

In [16]:
clean_and_append_special_2015(df)

,Month,Grade,Price
4,October 2014,AA,310.71
5,October 2014,AB,254.42
6,October 2014,C,232.12
20,November 2014,AA,300.48
21,November 2014,AB,251.45
22,November 2014,C,230.38
37,December 2014,AA,295.75
38,December 2014,AB,242.69
39,December 2014,C,224.97


### 20162017

In [17]:
df_2017 = read_pdf('pdfs/Per Month per Grade 20162017.pdf', pages='all', multiple_tables=True)

In [18]:
df = df_2017[1]

In [19]:
df

,0,1,2,3,4,5,6,7
0,Date By,NaN,Bags,Weight,Min,Max,VALUE,Avg
1,Month,Grade,Bought,Bought,PRICE,PRICE,US Dollar,PRICE
2,NaN,TT,472,"29,636",52,265,"129,405.32",218.32
3,NaN,UG1,"1,568","95,127",33,246,"319,449.16",167.91
4,NaN,UG2,229,"14,437",23,169,"26,846.68",92.98
5,Month Total,NaN,"27,179","1,666,383",23,443,"8,388,058.98",251.68
6,January 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,AA,"19,725","1,205,623",80,612,"8,434,398.96",349.79
8,NaN,AB,"39,640","2,405,751",64,520,"14,375,600.64",298.78
9,NaN,C,"13,001","798,625",50,335,"3,845,505.24",240.76


In [20]:
# Last 3 tables of df_2017 are well formed.
### First
df = df_2017[1]
df.columns = ['Month', 'Grade', 'Bags', 'Weight', 'Min', 'Max', 'USD', 'Price']
clean_and_append(df)

### Second
df = df_2017[2]
df.columns = ['Month', 'Grade', 'Bags', 'Weight', 'Min', 'Max', 'USD', 'Price']
clean_and_append(df)

### Third
df = df_2017[3]
df.columns = ['Month', 'Grade', 'Bags', 'Weight', 'Min', 'Max', 'USD', 'Price']
clean_and_append(df)

In [21]:
df = df_2017[0]

In [22]:
clean_and_append_special_2017(df)

,Month,0,Price
4,October 2016,AA,279.11
5,October 2016,AB,251.54
6,October 2016,C,216.86
21,November 2016,AA,292.50
22,November 2016,AB,266.95
23,November 2016,C,236.97
38,December 2016,AA,289.41
39,December 2016,AB,267.78
40,December 2016,C,241.44


### Check

In [23]:
df = pd.read_csv('monthly_coffee_prices.csv', names=['Month', 'Grade', 'Price'])

In [24]:
df

,Month,Grade,Price
1,October 2015,AA,236.98
2,October 2015,AB,211.06
3,October 2015,C,168.10
17,November 2015,AA,267.01
18,November 2015,AB,231.54
19,November 2015,C,163.36
34,December 2015,AA,300.68
35,December 2015,AB,258.07
36,December 2015,C,173.72
53,January 2016,AA,297.77
